# <font color="red">STEP 1: 文档提取测试</font>

In [ ]:
# 初始化
import django_setup

In [2]:
# 导入相关模型：get_user_model, Project, FileRecord, DocumentAnalysis, FileProjectLink, ProjectHistory
from django.contrib.auth import get_user_model
from apps.doc_analysis.models import DocumentAnalysis, InvalidStatusTransition
from apps.projects.models import Project
from apps.files.models import FileRecord
from django.core.files.uploadedfile import SimpleUploadedFile
from django.core.exceptions import ValidationError
from apps.doc_analysis.steps._01_extract_docx_elements import DocxExtractorStep
from apps.doc_analysis.pipeline.types import ModelData, DocxElements, OutlineAnalysisResult

In [ ]:
# 准备测试所需的 user, project, file_record  (其中project与file_record关联)
User = get_user_model()

# 获取已存在的测试数据

# 获取已存在的用户
user = User.objects.get(phone='18501771516')
print(f"用户: {user.phone}")
        
# 获取已存在的项目
project = Project.objects.get(project_name='测试项目1')
print(f"项目: {project.project_name}")
        
# 获取已存在的文件
file_record = FileRecord.objects.get(id='83db441f-1777-4505-bbfe-d9bf9be12451')
print(f"文件: {file_record.name}")

In [ ]:
# 清除‘测试分析A”，用于接下去的测试
DocumentAnalysis.objects.filter(title="测试分析A").delete()

In [ ]:
# 1. 创建新的文档分析实例 - 测试分析A
docx_analysis = DocumentAnalysis.objects.create(
    project=project,
    title="测试分析A",
    created_by=user,
    analysis_questions=["投标要求", "评分标准"]  # 示例分析问题
)
print(f"创建文档分析: {docx_analysis.title} (ID: {docx_analysis.id})")

In [ ]:
# 2.上传真实的 DOCX文件

# 2.1 准备文件路径
doc_path = "C:/Users/huiwa/Downloads/文本分析测试/CaseTest/case8：招标文件-第1包：一级压榨花生油.docx"

# 2.2 读取文件内容
with open(doc_path, 'rb') as f:
    file_content = f.read()
test_file = SimpleUploadedFile(
    "test_doc.docx",
    file_content,
    content_type="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
)
print(f"文件大小: {test_file.size}")

# 2. 创建新的文件记录 并 存储文件对象
new_file_record = FileRecord.objects.create(
    name="test_doc.docx",
    file=test_file,  # 使用之前准备的测试文件
    owner=user,
    size = test_file.size
)
print(f"创建文件记录: {new_file_record.name}")

In [7]:
# 3. 关联新文件
try:
    docx_analysis.update_file_record(new_file_record)
except Exception as e:
    print(f"关联文件失败: {str(e)}")

In [ ]:
# 4. 触发开始分析，并提取文档元素 elements 存入数据库
print("\n===== 最终状态检查 =====")
print(f"开始分析前-状态: {docx_analysis.status}")
#docx_analysis.start_analysis()
print(f"开始分析后-状态: {docx_analysis.status}")

# 初始化DocxExtractorStep
docx_extractor=DocxExtractorStep()
# 准备输入数据, 好比 DocumentAnalysis.instance.data
input_data = ModelData(model= DocumentAnalysis, instance=docx_analysis)
    
# 执行文档提取
docx_elements = docx_extractor.process(input_data)






In [ ]:
    # 打印提取结果
    print("文档提取成功！提取到的元素数量:", len(docx_elements))
    print("第一个元素示例:", docx_elements[0])
    
    # 检查保存到数据库的结果
    saved_analysis = DocumentAnalysis.objects.get(id=docx_analysis.id)
    print("保存的提取结果:", saved_analysis.extracted_elements)
    
except ValidationError as e:
    print("文档提取失败:", str(e))
    # 检查分析状态
    failed_analysis = DocumentAnalysis.objects.get(id=docx_analysis.id)
    print("分析状态:", failed_analysis.status)
    print("错误信息:", failed_analysis.error_message)

In [ ]:
print(type(docx_elements))

In [ ]:
print(type(docx_elements.elements[0]))

# <font color="red">STEP 2: outline 分析测试</font>

In [1]:
# 初始化
import django_setup

Development settings loaded
INSTALLED_APPS: ['django.contrib.admin', 'django.contrib.auth', 'django.contrib.contenttypes', 'django.contrib.sessions', 'django.contrib.messages', 'django.contrib.staticfiles', 'rest_framework', 'corsheaders', 'storages', 'apps.authentication', 'apps.files', 'apps.projects', 'apps.doc_analysis', 'apps.chat', 'django_filters', 'drf_spectacular', 'rest_framework_simplejwt.token_blacklist', 'django_celery_results', 'django_celery_beat']


INFO 2025-03-05 17:32:46,588 storage default_storage 的类型: COSStorage


Settings从哪里加载？: config.settings.development
项目根目录对么？: C:\Users\huiwa\Documents\_All_Projects\BidPilot_new\backend
文件存储settings对么？: apps.files.storage.COSStorage
文件default_storage对么？: COSStorage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- apps.projects
- apps.doc_analysis
- apps.chat
- django_filters
- drf_spectacular
- rest_framework_simplejwt.token_blacklist
- django_celery_results
- django_celery_beat


In [2]:
# 导入相关模型：get_user_model, Project, FileRecord, DocumentAnalysis, FileProjectLink, ProjectHistory
from django.contrib.auth import get_user_model
from apps.doc_analysis.models import DocumentAnalysis, InvalidStatusTransition
from apps.projects.models import Project
from apps.files.models import FileRecord
from django.core.files.uploadedfile import SimpleUploadedFile
from django.core.exceptions import ValidationError
from apps.doc_analysis.steps._01_extract_docx_elements import DocxExtractorStep
from apps.doc_analysis.pipeline.types import ModelData, DocxElements, OutlineAnalysisResult
from apps.doc_analysis.steps._02_outline_analysis import DocxOutlineAnalyzerStep
from pprint import pprint 

In [3]:
# 直接引用"测试分析A", 并获取其extracted_elements

saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
extracted_elements = saved_analysis.extracted_elements
pprint(extracted_elements['elements'][10])

{'content': '前附表',
 'is_toc': True,
 'position': 11,
 'toc_level': 2,
 'type': 'paragraph'}


In [4]:
# 创建 DocxElements 实例 from models.py 的 extracted_elements
docx_elements = DocxElements.from_model(extracted_elements)

In [5]:
print(docx_elements.format_heading_sections())

title:前附表, level:2, position:125
title:一、总则, level:2, position:130
title:二、招标文件, level:2, position:159
title:三、投标文件的编制, level:2, position:175
title:四、开标, level:2, position:256
title:五、评标, level:2, position:267
title:六、定标, level:2, position:294
title:七、合同授予, level:2, position:298
title:八、履约保证金, level:2, position:300
title:九、招标代理服务费的收取, level:2, position:302
title:附件一：资格审查表, level:2, position:352
title:附件二：符合性审查表, level:2, position:355
title:附件三：评分标准, level:2, position:357
title:附件 1 投标函, level:2, position:515
title:附件 2 投标保证书, level:2, position:532
title:附件 3 无违法记录声明, level:2, position:540
title:附件 4 法定代表人授权书, level:2, position:548
title:附件 5 投标人情况简介, level:2, position:556
title:附件 6 投标人 2021 年 1 月 1 日起至投标截止日止同类产品销售业绩表, level:2, position:559
title:附件 7 投标一览表, level:2, position:562
title:附件 8 分项报价表, level:2, position:570
title:附件 9 投标方案, level:2, position:579
title:附件 10 投标人资格声明, level:2, position:589
title:附件 11 资格证明文件, level:2, position:596
title:附件 12 与投标人存在“关系”的其他法人单位信息, level:2, pos

In [6]:
# 直接测试 outline_analyzer.py
outline_analyzer = DocxOutlineAnalyzerStep()


In [7]:
input_datas = outline_analyzer.prepare_requests_data(docx_elements)

In [8]:
print(type(input_datas))
print(input_datas[0])
print(input_datas[1])


<class 'list'>

## TOC TITLE LIST
title:第一章 招标公告, level:1, position:8
title:第二章 招标需求, level:1, position:9
title:第三章 投标人须知, level:1, position:10
title:第四章 评标办法及评分标准, level:1, position:21
title:第五章 合同条款及格式, level:1, position:25
title:第六章 投标文件格式, level:1, position:26

## HEADING TITLE LIST
title:北京京铁运恒采购供应站有限公司 2024 年端午节物资采购项目, level:1, position:1
title:招标文件, level:1, position:2
title:第一章 招标公告, level:1, position:46
title:第二章 招标需求, level:1, position:95
title:第三章 投标人须知, level:1, position:124
title:第四章 评标办法及评分标准, level:1, position:308
title:第五章 合同条款及格式, level:1, position:366
title:第六章 投标文件格式, level:1, position:501


## TOC TITLE LIST
title:前附表, level:2, position:11
title:一、总则, level:2, position:12
title:二、招标文件, level:2, position:13
title:三、投标文件的编制, level:2, position:14
title:四、开标, level:2, position:15
title:五、评标, level:2, position:16
title:六、定标, level:2, position:17
title:七、合同授予, level:2, position:18
title:八、履约保证金, level:2, position:19
title:九、招标代理服务费的收取, level:2, position:20
title:附件一：资格审查表

In [9]:
simulated_prompt, formatted_messages = outline_analyzer.simulate_prompt(input_datas[0])
pprint(formatted_messages)


[{'content': '你是一个专业的招标文档分析助手，帮助用户分析文档的结构和内容。', 'role': 'system'},
 {'content': '\n'
             '# Task\n'
             '分析招标文档的目录结构和正文标题之间的一致性\n'
             '\n'
             '# Requirements\n'
             '- 比对目录中的标题和正文中的实际标题\n'
             '- 忽略标点符号和空格的差异\n'
             '- 仅匹配标题的实际文本内容\n'
             '- 分别罗列出"目录中存在但正文中不存在"和"正文中存在但目录中不存在"的标题\n'
             '\n'
             '# Output\n'
             '## Rules\n'
             '- 只输出JSON格式的结果\n'
             '- 不使用Markdown格式\n'
             '- 确保JSON格式严格有效\n'
             '- 空元素使用[]\n'
             '\n'
             '## Format\n'
             '\n'
             '请严格按照以下JSON格式输出目录分析结果，不要包含任何额外的解释或说明：\n'
             '{\n'
             '    "toc_only_titles": [\n'
             '        {\n'
             '            "title": "标题内容",\n'
             '            "position": "目录中的位置",\n'
             '            "level": "标题层级"\n'
             '        }\n'
             '    ],\n'
             '    "heading_only_titles": [\n'
    

In [10]:
analysis_result = outline_analyzer.process(docx_elements)

TypeError: BatchLLMService.batch_process() got an unexpected keyword argument 'parallel'

In [ ]:
# 打印输出的结果
print(type(analysis_result))
print(type(analysis_result.analysis_result))
print(type(analysis_result.analysis_result.result))
print(type(analysis_result.analysis_result.result[0]))
pprint(analysis_result)


In [ ]:
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
pprint(saved_analysis.outline_analysis_result)


# <font color="red">STEP 3: 测试outline_improvement.py</font>


In [ ]:
# 初始化
import django_setup

In [2]:
# 导入相关模型
from django.contrib.auth import get_user_model
from apps.doc_analysis.models import DocumentAnalysis, InvalidStatusTransition
from apps.projects.models import Project
from apps.files.models import FileRecord
from django.core.files.uploadedfile import SimpleUploadedFile
from django.core.exceptions import ValidationError
from apps.doc_analysis.steps._01_extract_docx_elements import DocxExtractorStep
from apps.doc_analysis.pipeline.types import ModelData, DocxElements, OutlineAnalysisResult
from apps.doc_analysis.steps._02_outline_analysis import DocxOutlineAnalyzerStep
from apps.doc_analysis.steps._03_outline_improvement import OutlineImprovementStep
from pprint import pprint 

In [3]:
# 直接引用"测试分析A", 并获取其docx_elements 和 outline_analysis_result

saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
docx_elements = DocxElements.from_model(saved_analysis.extracted_elements)
outline_analysis_result = OutlineAnalysisResult.from_model(saved_analysis.outline_analysis_result)

In [ ]:
# 打印输出的结果
print(type(outline_analysis_result.analysis_result))
print(type(outline_analysis_result.analysis_result.result))
print(type(outline_analysis_result.analysis_result.result[0]))
pprint(outline_analysis_result)


## 分步骤测试

In [5]:
# 构建实例
outline_improver = OutlineImprovementStep()

In [ ]:
# titles 提取
titles_to_improve = outline_improver.extract_titles_from_analysis(outline_analysis_result)
pprint(titles_to_improve)


In [8]:
# 改进元素
improved_elements = outline_improver.improve_document_elements(docx_elements, titles_to_improve)

In [ ]:
# 打印改进后的数据
pprint(improved_elements)

## 集成测试

In [ ]:
# 模拟用户确认
outline_analysis_result.user_confirm = True

pprint(outline_analysis_result)


In [6]:
# 完整测试 _02_outline_analysis.py
outline_improver = OutlineImprovementStep()
improved_docx_elements = outline_improver.process((docx_elements, outline_analysis_result))

In [ ]:
# 打印原来的数据
pprint(docx_elements)

In [ ]:
# 打印改进后的数据
pprint(improved_docx_elements)

# <font color=red>STEP 4: 构建文档树DocxTree <font>


In [ ]:
# 初始化
import django_setup

In [2]:
# show_structure函数（）
from pprint import pprint
from typing import Any

def show_structure(data: Any, name: str = "data", max_depth: int = None) -> None:
    """
    显示数据的类型和结构的辅助函数
    
    参数:
        data: 要检查的数据
        name: 数据的名称/标签
        max_depth: pprint的最大深度
    """
    print(f"\n{'='*50}")
    print(f"检查对象: {name}")
    print(f"类型: {type(data)}")
    
    if hasattr(data, '__len__'):
        print(f"长度: {len(data)}")
    
    print("\n结构:")
    pprint(data, depth=max_depth, width=100)
    print(f"{'='*50}\n")


In [3]:
# 导入相关模型
from apps.doc_analysis.models import DocumentAnalysis
from apps.doc_analysis.pipeline.types import ModelData, ImprovedDocxElements, SimpleDocxNode, DocxTree
from apps.doc_analysis.steps._04_build_docxtree import BuildDocxTree
from pprint import pprint 

In [4]:
# 直接引用"测试分析A", 并获取 improved_docx_elements
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
improved_docx_elements = ImprovedDocxElements.from_model(saved_analysis.improved_docx_elements)
#elements = improved_docx_elements.elements
#show_structure(elements)


## 构建文档树

In [ ]:
# 构建文档树
build_docx_tree = BuildDocxTree()
docx_tree = build_docx_tree.process(improved_docx_elements)
show_structure(docx_tree)


In [ ]:
# 按顺序打印所有节点
pprint(docx_tree._ordered_nodes)

In [ ]:
# 按树结构打印所有节点
docx_tree.print_tree()

In [ ]:
print(docx_tree.format_titles())

In [ ]:
# 检查文档树在models.py里的存储情况
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
print(type(saved_analysis.docxtree))
pprint(saved_analysis.docxtree)


## 测试从数据库提取文档树

In [ ]:
# 还原：从models.py存储到doc_tree类型的还原
docx_tree_from_model = DocxTree.from_model(saved_analysis.docxtree)
print(type(docx_tree_from_model))
pprint(docx_tree_from_model)

In [ ]:
# 还原 ： 按顺序打印所有节点
pprint(docx_tree_from_model._ordered_nodes)

In [ ]:
# 还原： 按树结构打印所有节点
docx_tree_from_model.print_tree()

In [ ]:
print(docx_tree_from_model.format_titles())

## 测试文档树方法：查找节点，打印树，标题格式化

In [ ]:
# 查找某个节点 find_node()， 测试 .find_node() 方法，比如node_id = 78
node = docx_tree.find_node(78)
pprint(node)

In [ ]:
# 打印某个节点以下的文档树结构
docx_tree.print_tree(124)


In [ ]:
# 格式化 标题结构 用于大模型分析
titles_nodes = docx_tree.format_titles()
pprint(titles_nodes)

## 测试添加节点

In [ ]:
# 添加 节点
docx_tree.add_title_node(
    content = '>>>> 插入新节点',
    level = 2,
    after_node_id = 150 
)

In [ ]:
# 打印结果 - 看顺序列表里是否有插入？
pprint(docx_tree._ordered_nodes[140:160])

In [ ]:
# 打印文档树
docx_tree.print_tree(124)

## 测试调整节点类型

In [19]:
# 节点改变层级
docx_tree.convert_to_title_node(156,2)

In [ ]:
# 打印结果 - 看顺序列表里是否有插入？
pprint(docx_tree._ordered_nodes[140:160])

In [ ]:
# 打印文档树
docx_tree.print_tree(124)

# <font color=red>  STEP 5: 测试文档树的使用</font>

In [ ]:
# 初始化
import django_setup

In [2]:
# 导入相关模型
from apps.doc_analysis.models import DocumentAnalysis
from apps.doc_analysis.pipeline.types import ModelData,DocxTree, DocxTreeMoreTitles
from apps.doc_analysis.steps._04_build_docxtree import BuildDocxTree
from apps.doc_analysis.steps._05_more_subtitles import MoreSubTitlesStep
from pprint import pprint 

In [ ]:
# 直接引用"测试分析A", 并获取 doc_tree
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
docx_tree = DocxTree.from_model(saved_analysis.docxtree)
print(type(docx_tree))
print(docx_tree.format_titles())

In [ ]:
leaf_titles = docx_tree.get_leaf_titles()
pprint(leaf_titles)

In [ ]:
# 创建子标题分析 实例
subtitles_analyzer = MoreSubTitlesStep()
# 检查 data_inputs
data_inputs = subtitles_analyzer.prepare_requests_data(docx_tree)
# 检查prompt
simulated_prompt, formatted_messages = subtitles_analyzer.simulate_prompt(data_inputs[0])
print(formatted_messages)

In [ ]:
analysis_reuslts = subtitles_analyzer.process(docx_tree)

In [ ]:
print(type(analysis_reuslts))
pprint(analysis_reuslts)

In [ ]:
# 检查 more_subtitles 在models.py里的存储情况
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
print(type(saved_analysis.more_subtitles))
pprint(saved_analysis.more_subtitles)

In [13]:
# 测试 从models.py提取 more_subtitles, 并转换为DocxTreeMoreTitles格式
more_subtitles_from_model = DocxTreeMoreTitles.from_model(saved_analysis.more_subtitles)
print(type(more_subtitles_from_model))
pprint(more_subtitles_from_model)